Crawl link bài báo theo ngày

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import json
import os
import calendar # Import thư viện calendar để lấy số ngày trong tháng
import re # Import regex để chuẩn hóa ngày

In [16]:
def navigate_and_select_date_cafef(driver, url, target_day_value, target_month_value, target_year_value, css_selector_to_click_init):
    """
    Truy cập trang web, click phần tử ban đầu, chọn ngày/tháng/năm trong dropdowns,
    và click nút XEM. Driver được truyền vào và trả về để tái sử dụng.
    """
    try:
        if driver.current_url != url or not driver.page_source:
            print(f"Đang truy cập: {url}")
            driver.get(url)
            time.sleep(5)

        # Chỉ click nếu phần tử này xuất hiện và cần click lại sau khi thay đổi ngày
        try:
            element_init_click = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector_to_click_init))
            )
            # Kiểm tra để tránh click lại nếu không cần
            # Ví dụ: if "active" not in element_init_click.get_attribute("class"):
            element_init_click.click()
            time.sleep(3)
            print("Đã click vào phần tử ban đầu (nếu cần).")
        except Exception:
            print(f"Không cần click hoặc không tìm thấy phần tử ban đầu '{css_selector_to_click_init}'.")
            pass # Bỏ qua nếu không tìm thấy hoặc không click được

        # Tương tác với dropdown Năm
        select_element_year = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "opNam"))
        )
        select_year = Select(select_element_year)
        select_year.select_by_value(str(target_year_value))
        print(f"Đã chọn Năm: {target_year_value}.")
        time.sleep(1)

        # Tương tác với dropdown Tháng
        select_element_month = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "opThang"))
        )
        select_month = Select(select_element_month)
        select_month.select_by_value(str(target_month_value))
        print(f"Đã chọn Tháng: {target_month_value}.")
        time.sleep(1)

        # Tương tác với dropdown Ngày
        select_element_day = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "opNgay"))
        )
        select_day = Select(select_element_day)
        select_day.select_by_value(str(target_day_value))
        print(f"Đã chọn Ngày: {target_day_value}.")
        time.sleep(1)
        
        # Click vào nút "XEM" để áp dụng bộ lọc ngày
        btn_view_selector = "a.btn-view.font1"
        print(f"Đang tìm và click nút 'XEM' với selector: {btn_view_selector}")
        btn_view = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, btn_view_selector))
        )
        btn_view.click()
        print("Đã click nút 'XEM'.")
        time.sleep(5) 

        return driver
    
    except Exception as e:
        print(f"Lỗi trong navigate_and_select_date_cafef: {e}")
        return None

In [17]:
def crawl_article_urls_from_filtered_page(driver_instance, article_container_selector, title_css_inside_container, sapo_css_inside_container, article_url_selector, page_date_selector):
    """
    Cào URL, tiêu đề, mô tả các bài báo từ trang hiện tại của driver sau khi đã lọc.
    Cũng lấy ngày hiển thị trên tiêu đề trang để tổ chức dữ liệu.
    """
    articles_list_for_date = []
    
    # Cuộn xuống cuối trang để tải hết nội dung (nếu có lazy loading)
    print("Đang cuộn xuống cuối trang để tải hết nội dung...")
    last_height = driver_instance.execute_script("return document.body.scrollHeight")
    while True:
        driver_instance.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) # Chờ tải nội dung
        new_height = driver_instance.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    print("Đã cuộn đến cuối trang.")

    html_content = driver_instance.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    # Lấy ngày từ tiêu đề trang (selector bạn cung cấp)
    page_date_element = soup.select_one(page_date_selector)
    page_date_text = page_date_element.get_text(strip=True) if page_date_element else "Unknown Date"
    
    date_match = re.search(r'\d{1,2}/\d{1,2}/\d{4}', page_date_text)
    if date_match:
        formatted_date = date_match.group(0).replace('/', '-') # Chuyển 1/6/2025 thành 01-06-2025
    else:
        formatted_date = page_date_text.replace(':', '').replace('/', '-').replace(' ', '_') 
    
    print(f"Ngày của trang được cào (để làm key JSON): {formatted_date}")
    
    article_containers = soup.select(article_container_selector)
    print(f"Tìm thấy {len(article_containers)} khối bài báo.")

    if not article_containers:
        print("Không tìm thấy bài báo nào với selector đã cho. Vui lòng kiểm tra lại selector.")
        return formatted_date, []

    for i, container in enumerate(article_containers):
        title_element = container.select_one(title_css_inside_container)
        sapo_element = container.select_one(sapo_css_inside_container)
        article_url_element = container.select_one(article_url_selector) 

        title = title_element.get_text(strip=True) if title_element else None
        url = article_url_element['href'] if article_url_element and 'href' in article_url_element.attrs else None
        sapo = sapo_element.get_text(strip=True) if sapo_element else None

        if title and url:
            if url and not url.startswith('http'):
                url = f"https://cafef.vn{url}" 
            
            articles_list_for_date.append({
                'Tieu de': title,
                'URL': url,
                'Mo ta': sapo
            })
        else:
            print(f"Bỏ qua bài báo {i+1} do thiếu tiêu đề hoặc URL.")
            pass # Giảm bớt log lỗi nếu có nhiều bài thiếu

    return formatted_date, articles_list_for_date

In [6]:
# --- Hàm điều phối để cào dữ liệu của cả tháng ---
def crawl_month_data(base_url, init_click_selector, target_month, target_year, 
                     article_container_css, title_css, sapo_css, article_url_css, page_date_css, 
                     output_json_filename='raw_news_urls.json'):
    """
    Cào dữ liệu URL/mô tả của các bài báo cho từng ngày trong một tháng cụ thể.
    Lưu dữ liệu vào file JSON.
    """
    all_crawled_data = {}
    if os.path.exists(output_json_filename):
        with open(output_json_filename, 'r', encoding='utf-8') as f:
            try:
                all_crawled_data = json.load(f)
                print(f"Đã tải dữ liệu cũ từ '{output_json_filename}'")
            except json.JSONDecodeError:
                print(f"Lỗi đọc file JSON '{output_json_filename}', tạo file mới.")
                all_crawled_data = {}

    driver = None
    try:
        driver = webdriver.Chrome()

        # Lấy số ngày trong tháng đó
        num_days = calendar.monthrange(target_year, target_month)[1]
        print(f"Đang tiến hành cào dữ liệu cho tháng {target_month}/{target_year} ({num_days} ngày).")

        for day in range(1, num_days + 1):
            day_str = str(day)
            print(f"\n--- Đang xử lý ngày: {day_str}/{target_month}/{target_year} ---")
            
            # Chọn ngày/tháng/năm và click "XEM"
            # driver được truyền vào và tái sử dụng
            current_driver = navigate_and_select_date_cafef(
                driver, base_url, day_str, target_month, target_year, init_click_selector
            )

            if current_driver:
                current_date_key, crawled_articles = crawl_article_urls_from_filtered_page(
                    current_driver, article_container_css, title_css, sapo_css, article_url_css, page_date_css
                )
                
                if crawled_articles:
                    all_crawled_data[current_date_key] = crawled_articles
                    print(f"Đã cào được {len(crawled_articles)} bài báo cho ngày '{current_date_key}'.")
                    # Lưu lại file JSON sau mỗi ngày để tránh mất dữ liệu nếu script dừng đột ngột
                    with open(output_json_filename, 'w', encoding='utf-8') as f:
                        json.dump(all_crawled_data, f, ensure_ascii=False, indent=4)
                    print(f"Đã cập nhật '{output_json_filename}'.")
                else:
                    print(f"Không có URL bài báo nào được cào về cho ngày '{current_date_key}'.")
            else:
                print(f"Bỏ qua ngày {day_str}/{target_month}/{target_year} do lỗi điều hướng/lọc.")

    except Exception as e:
        print(f"Đã xảy ra lỗi tổng thể trong quá trình cào tháng: {e}")
    finally:
        if driver:
            print("Đang đóng trình duyệt.")
            driver.quit() # Đóng driver khi hoàn tất hoặc có lỗi

# --- Cấu hình các Selector và bắt đầu crawl ---
if __name__ == "__main__":
    target_base_url = "https://cafef.vn/thi-truong-chung-khoan.chn"
    
    # CSS Selector của phần tử ban đầu bạn muốn click (tiêu đề bài nổi bật)
    css_selector_of_initial_element_to_click = "#admWrapsite > div.main > div.list-section > div > div.list-main > div.noibat_cate > div.list-focus-main > div.firstitem.wp100.clearfix > div > h2 > a"
    
    # --- CÁC SELECTOR ĐỂ CÀO NỘI DUNG BÀI BÁO (URL/Mô tả) ---
    article_container_css = "#hasWechoice > div" 
    title_css_inside_container = "h3 > a" 
    sapo_css_inside_container = "p.sapo"
    article_url_css_inside_container = "a.avatar.img-resize"
    page_date_css = "#admWrapsite > div.main > div.list-section.list-event > div > div.list-main > h1 > a"

    # --- tháng và năm cần cào ---
    month_to_crawl = 6 
    year_to_crawl = 2025 

    crawl_month_data(target_base_url, 
                     css_selector_of_initial_element_to_click, 
                     month_to_crawl, 
                     year_to_crawl,
                     article_container_css,
                     title_css_inside_container,
                     sapo_css_inside_container,
                     article_url_css_inside_container,
                     page_date_css)

Đã tải dữ liệu cũ từ 'raw_news_urls.json'
Đang tiến hành cào dữ liệu cho tháng 6/2025 (30 ngày).

--- Đang xử lý ngày: 1/6/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 6.
Đã chọn Ngày: 1.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 1-6-2025
Tìm thấy 8 khối bài báo.
Đã cào được 7 bài báo cho ngày '1-6-2025'.
Đã cập nhật 'raw_news_urls.json'.

--- Đang xử lý ngày: 2/6/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 6.
Đã chọn Ngày: 2.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 2-6-2025
Tìm thấy 20 khố

In [7]:
# --- Cấu hình các Selector và bắt đầu crawl ---
if __name__ == "__main__":
    target_base_url = "https://cafef.vn/thi-truong-chung-khoan.chn"
    
    # CSS Selector của phần tử ban đầu bạn muốn click (tiêu đề bài nổi bật)
    css_selector_of_initial_element_to_click = "#admWrapsite > div.main > div.list-section > div > div.list-main > div.noibat_cate > div.list-focus-main > div.firstitem.wp100.clearfix > div > h2 > a"
    
    # --- CÁC SELECTOR ĐỂ CÀO NỘI DUNG BÀI BÁO (URL/Mô tả) ---
    article_container_css = "#hasWechoice > div" 
    title_css_inside_container = "h3 > a" 
    sapo_css_inside_container = "p.sapo"
    article_url_css_inside_container = "a.avatar.img-resize"
    page_date_css = "#admWrapsite > div.main > div.list-section.list-event > div > div.list-main > h1 > a"

    # --- tháng và năm cần cào ---
    month_to_crawl = 5 
    year_to_crawl = 2025 

    crawl_month_data(target_base_url, 
                     css_selector_of_initial_element_to_click, 
                     month_to_crawl, 
                     year_to_crawl,
                     article_container_css,
                     title_css_inside_container,
                     sapo_css_inside_container,
                     article_url_css_inside_container,
                     page_date_css)

Đã tải dữ liệu cũ từ 'raw_news_urls.json'
Đang tiến hành cào dữ liệu cho tháng 5/2025 (31 ngày).

--- Đang xử lý ngày: 1/5/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 5.
Đã chọn Ngày: 1.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 1-5-2025
Tìm thấy 11 khối bài báo.
Đã cào được 10 bài báo cho ngày '1-5-2025'.
Đã cập nhật 'raw_news_urls.json'.

--- Đang xử lý ngày: 2/5/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 5.
Đã chọn Ngày: 2.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 2-5-2025
Tìm thấy 10 k

In [8]:
# --- Cấu hình các Selector và bắt đầu crawl ---
if __name__ == "__main__":
    target_base_url = "https://cafef.vn/thi-truong-chung-khoan.chn"
    
    # CSS Selector của phần tử ban đầu bạn muốn click (tiêu đề bài nổi bật)
    css_selector_of_initial_element_to_click = "#admWrapsite > div.main > div.list-section > div > div.list-main > div.noibat_cate > div.list-focus-main > div.firstitem.wp100.clearfix > div > h2 > a"
    
    # --- CÁC SELECTOR ĐỂ CÀO NỘI DUNG BÀI BÁO (URL/Mô tả) ---
    article_container_css = "#hasWechoice > div" 
    title_css_inside_container = "h3 > a" 
    sapo_css_inside_container = "p.sapo"
    article_url_css_inside_container = "a.avatar.img-resize"
    page_date_css = "#admWrapsite > div.main > div.list-section.list-event > div > div.list-main > h1 > a"

    # --- tháng và năm cần cào ---
    month_to_crawl = 4 
    year_to_crawl = 2025 

    crawl_month_data(target_base_url, 
                     css_selector_of_initial_element_to_click, 
                     month_to_crawl, 
                     year_to_crawl,
                     article_container_css,
                     title_css_inside_container,
                     sapo_css_inside_container,
                     article_url_css_inside_container,
                     page_date_css)

Đã tải dữ liệu cũ từ 'raw_news_urls.json'
Đang tiến hành cào dữ liệu cho tháng 4/2025 (30 ngày).

--- Đang xử lý ngày: 1/4/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 4.
Đã chọn Ngày: 1.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 1-4-2025
Tìm thấy 20 khối bài báo.
Đã cào được 19 bài báo cho ngày '1-4-2025'.
Đã cập nhật 'raw_news_urls.json'.

--- Đang xử lý ngày: 2/4/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 4.
Đã chọn Ngày: 2.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 2-4-2025
Tìm thấy 20 k

In [9]:
# --- Cấu hình các Selector và bắt đầu crawl ---
if __name__ == "__main__":
    target_base_url = "https://cafef.vn/thi-truong-chung-khoan.chn"
    
    # CSS Selector của phần tử ban đầu bạn muốn click (tiêu đề bài nổi bật)
    css_selector_of_initial_element_to_click = "#admWrapsite > div.main > div.list-section > div > div.list-main > div.noibat_cate > div.list-focus-main > div.firstitem.wp100.clearfix > div > h2 > a"
    
    # --- CÁC SELECTOR ĐỂ CÀO NỘI DUNG BÀI BÁO (URL/Mô tả) ---
    article_container_css = "#hasWechoice > div" 
    title_css_inside_container = "h3 > a" 
    sapo_css_inside_container = "p.sapo"
    article_url_css_inside_container = "a.avatar.img-resize"
    page_date_css = "#admWrapsite > div.main > div.list-section.list-event > div > div.list-main > h1 > a"

    # --- tháng và năm cần cào ---
    month_to_crawl = 3 
    year_to_crawl = 2025 

    crawl_month_data(target_base_url, 
                     css_selector_of_initial_element_to_click, 
                     month_to_crawl, 
                     year_to_crawl,
                     article_container_css,
                     title_css_inside_container,
                     sapo_css_inside_container,
                     article_url_css_inside_container,
                     page_date_css)

Đã tải dữ liệu cũ từ 'raw_news_urls.json'
Đang tiến hành cào dữ liệu cho tháng 3/2025 (31 ngày).

--- Đang xử lý ngày: 1/3/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 3.
Đã chọn Ngày: 1.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 1-3-2025
Tìm thấy 11 khối bài báo.
Đã cào được 10 bài báo cho ngày '1-3-2025'.
Đã cập nhật 'raw_news_urls.json'.

--- Đang xử lý ngày: 2/3/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 3.
Đã chọn Ngày: 2.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 2-3-2025
Tìm thấy 10 k

In [10]:
# --- Cấu hình các Selector và bắt đầu crawl ---
if __name__ == "__main__":
    target_base_url = "https://cafef.vn/thi-truong-chung-khoan.chn"
    
    # CSS Selector của phần tử ban đầu bạn muốn click (tiêu đề bài nổi bật)
    css_selector_of_initial_element_to_click = "#admWrapsite > div.main > div.list-section > div > div.list-main > div.noibat_cate > div.list-focus-main > div.firstitem.wp100.clearfix > div > h2 > a"
    
    # --- CÁC SELECTOR ĐỂ CÀO NỘI DUNG BÀI BÁO (URL/Mô tả) ---
    article_container_css = "#hasWechoice > div" 
    title_css_inside_container = "h3 > a" 
    sapo_css_inside_container = "p.sapo"
    article_url_css_inside_container = "a.avatar.img-resize"
    page_date_css = "#admWrapsite > div.main > div.list-section.list-event > div > div.list-main > h1 > a"

    # --- tháng và năm cần cào ---
    month_to_crawl = 2 
    year_to_crawl = 2025 

    crawl_month_data(target_base_url, 
                     css_selector_of_initial_element_to_click, 
                     month_to_crawl, 
                     year_to_crawl,
                     article_container_css,
                     title_css_inside_container,
                     sapo_css_inside_container,
                     article_url_css_inside_container,
                     page_date_css)

Đã tải dữ liệu cũ từ 'raw_news_urls.json'
Đang tiến hành cào dữ liệu cho tháng 2/2025 (28 ngày).

--- Đang xử lý ngày: 1/2/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 2.
Đã chọn Ngày: 1.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 1-2-2025
Tìm thấy 4 khối bài báo.
Đã cào được 3 bài báo cho ngày '1-2-2025'.
Đã cập nhật 'raw_news_urls.json'.

--- Đang xử lý ngày: 2/2/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 2.
Đã chọn Ngày: 2.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 2-2-2025
Tìm thấy 7 khối

In [11]:
# --- Cấu hình các Selector và bắt đầu crawl ---
if __name__ == "__main__":
    target_base_url = "https://cafef.vn/thi-truong-chung-khoan.chn"
    
    # CSS Selector của phần tử ban đầu bạn muốn click (tiêu đề bài nổi bật)
    css_selector_of_initial_element_to_click = "#admWrapsite > div.main > div.list-section > div > div.list-main > div.noibat_cate > div.list-focus-main > div.firstitem.wp100.clearfix > div > h2 > a"
    
    # --- CÁC SELECTOR ĐỂ CÀO NỘI DUNG BÀI BÁO (URL/Mô tả) ---
    article_container_css = "#hasWechoice > div" 
    title_css_inside_container = "h3 > a" 
    sapo_css_inside_container = "p.sapo"
    article_url_css_inside_container = "a.avatar.img-resize"
    page_date_css = "#admWrapsite > div.main > div.list-section.list-event > div > div.list-main > h1 > a"

    # --- tháng và năm cần cào ---
    month_to_crawl = 1 
    year_to_crawl = 2025 

    crawl_month_data(target_base_url, 
                     css_selector_of_initial_element_to_click, 
                     month_to_crawl, 
                     year_to_crawl,
                     article_container_css,
                     title_css_inside_container,
                     sapo_css_inside_container,
                     article_url_css_inside_container,
                     page_date_css)

Đã tải dữ liệu cũ từ 'raw_news_urls.json'
Đang tiến hành cào dữ liệu cho tháng 1/2025 (31 ngày).

--- Đang xử lý ngày: 1/1/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 1.
Đã chọn Ngày: 1.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 1-1-2025
Tìm thấy 7 khối bài báo.
Đã cào được 6 bài báo cho ngày '1-1-2025'.
Đã cập nhật 'raw_news_urls.json'.

--- Đang xử lý ngày: 2/1/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 1.
Đã chọn Ngày: 2.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 2-1-2025
Tìm thấy 20 khố

Crawl nội dung bài báo theo ngày

In [24]:
import json
import os
from datetime import date, timedelta

def find_missing_dates_in_json(json_file_path='raw_news_urls.json', start_date_str='01-01-2025', end_date_str='23-06-2025'):
    """
    Kiểm tra file JSON để tìm các ngày bị thiếu dữ liệu trong một khoảng thời gian nhất định.

    Args:
        json_file_path (str): Đường dẫn đến file JSON chứa dữ liệu đã cào.
        start_date_str (str): Ngày bắt đầu của khoảng thời gian kiểm tra (định dạng DD-MM-YYYY).
        end_date_str (str): Ngày kết thúc của khoảng thời gian kiểm tra (định dạng DD-MM-YYYY).
    """
    
    # 1. Đọc dữ liệu từ file JSON
    crawled_dates = set()
    if os.path.exists(json_file_path):
        with open(json_file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                for date_key in data.keys():
                    # Chuẩn hóa key từ JSON về định dạng DD-MM-YYYY (thêm số 0 nếu cần)
                    parts = date_key.split('-')
                    if len(parts) == 3:
                        try:
                            day = int(parts[0])
                            month = int(parts[1])
                            year = int(parts[2])
                            # Định dạng lại thành DD-MM-YYYY
                            formatted_date = f"{day:02d}-{month:02d}-{year}"
                            crawled_dates.add(formatted_date)
                        except ValueError:
                            print(f"Cảnh báo: Định dạng ngày không hợp lệ trong JSON key: {date_key}")
                    else:
                        print(f"Cảnh báo: Định dạng ngày không mong muốn trong JSON key: {date_key}")
        
            except json.JSONDecodeError:
                print(f"Lỗi: Không thể đọc file JSON '{json_file_path}'. File có thể bị hỏng hoặc trống.")
                return

    print(f"Số ngày đã cào được trong file JSON: {len(crawled_dates)}")

    # 2. Tạo danh sách tất cả các ngày dự kiến trong khoảng
    # Chuyển đổi chuỗi ngày thành đối tượng date
    start_date = date.fromisoformat(start_date_str.replace('-', '/').split('/')[2] + '-' + \
                                    start_date_str.replace('-', '/').split('/')[1] + '-' + \
                                    start_date_str.replace('-', '/').split('/')[0])
    end_date = date.fromisoformat(end_date_str.replace('-', '/').split('/')[2] + '-' + \
                                  end_date_str.replace('-', '/').split('/')[1] + '-' + \
                                  end_date_str.replace('-', '/').split('/')[0])

    expected_dates = set()
    current_date = start_date
    while current_date <= end_date:
        # Chuẩn hóa ngày dự kiến về định dạng DD-MM-YYYY
        expected_dates.add(current_date.strftime("%d-%m-%Y"))
        current_date += timedelta(days=1)
    
    print(f"Tổng số ngày dự kiến cần có: {len(expected_dates)}")

    # 3. So sánh để tìm các ngày còn thiếu
    missing_dates = sorted(list(expected_dates - crawled_dates))

    # 4. In ra những ngày còn thiếu
    if missing_dates:
        print("\n--- Các ngày còn thiếu dữ liệu trong file JSON ---")
        for m_date in missing_dates:
            print(m_date)
        print(f"\nTổng cộng có {len(missing_dates)} ngày bị thiếu.")
    else:
        print("\nKhông có ngày nào bị thiếu dữ liệu trong khoảng đã chỉ định.")

In [4]:
if __name__ == "__main__":
    json_file = 'raw_news_urls.json' 
    
    # Đặt khoảng thời gian
    start_check_date = '01-01-2025'
    end_check_date = '23-06-2025' 

    find_missing_dates_in_json(json_file, start_check_date, end_check_date)

Số ngày đã cào được trong file JSON: 160
Tổng số ngày dự kiến cần có: 174

--- Các ngày còn thiếu dữ liệu trong file JSON ---
03-02-2025
04-01-2025
07-03-2025
10-03-2025
11-02-2025
14-02-2025
14-03-2025
16-04-2025
17-01-2025
18-04-2025
20-02-2025
25-03-2025
28-02-2025
29-01-2025

Tổng cộng có 14 ngày bị thiếu.


In [5]:
# --- Hàm để cào các ngày bị thiếu ---
def crawl_missing_dates(list_of_missing_dates_str, base_url, init_click_selector, 
                       article_container_css, title_css, sapo_css, article_url_css, page_date_css, 
                       output_json_filename='raw_news_urls.json'):
    """
    Cào dữ liệu URL/mô tả của các bài báo cho các ngày cụ thể trong danh sách bị thiếu.
    Cập nhật dữ liệu vào file JSON hiện có.

    Args:
        list_of_missing_dates_str (list): Danh sách các chuỗi ngày bị thiếu (định dạng DD-MM-YYYY).
        ... các tham số selector và file khác ...
    """
    
    if not list_of_missing_dates_str:
        print("Không có ngày nào cần cào lại.")
        return

    all_crawled_data = {}
    if os.path.exists(output_json_filename):
        with open(output_json_filename, 'r', encoding='utf-8') as f:
            try:
                all_crawled_data = json.load(f)
                print(f"Đã tải dữ liệu cũ từ '{output_json_filename}' để cập nhật.")
            except json.JSONDecodeError:
                print(f"Lỗi đọc file JSON '{output_json_filename}'. Tạo dữ liệu mới.")
                all_crawled_data = {}
    else:
        print(f"File '{output_json_filename}' không tồn tại. Tạo file mới.")

    driver = None
    try:
        driver = webdriver.Chrome() # Khởi tạo driver một lần
        print(f"\n--- Bắt đầu cào lại {len(list_of_missing_dates_str)} ngày bị thiếu ---")

        for date_str_ddmmyyyy in list_of_missing_dates_str:
            parts = date_str_ddmmyyyy.split('-')
            if len(parts) == 3:
                day = int(parts[0])
                month = int(parts[1])
                year = int(parts[2])
            else:
                print(f"Cảnh báo: Định dạng ngày '{date_str_ddmmyyyy}' không hợp lệ. Bỏ qua.")
                continue
            
            print(f"\n--- Đang xử lý ngày thiếu: {day}/{month}/{year} ---")
            
            current_driver = navigate_and_select_date_cafef(
                driver, base_url, str(day), str(month), str(year), init_click_selector
            )

            if current_driver:
                current_date_key_formatted, crawled_articles = crawl_article_urls_from_filtered_page(
                    current_driver, article_container_css, title_css, sapo_css, article_url_css, page_date_css
                )
                
                if crawled_articles:
                    all_crawled_data[current_date_key_formatted] = crawled_articles
                    print(f"Đã cào được {len(crawled_articles)} bài báo cho ngày '{current_date_key_formatted}'.")
                    
                    # Lưu lại file JSON sau mỗi ngày cào thành công để đảm bảo cập nhật
                    with open(output_json_filename, 'w', encoding='utf-8') as f:
                        json.dump(all_crawled_data, f, ensure_ascii=False, indent=4)
                    print(f"Đã cập nhật '{output_json_filename}'.")
                else:
                    print(f"Không có URL bài báo nào được cào về cho ngày '{current_date_key_formatted}'.")
            else:
                print(f"Bỏ qua ngày {day}/{month}/{year} do lỗi điều hướng/lọc.")
    
    except Exception as e:
        print(f"Đã xảy ra lỗi tổng thể trong quá trình cào các ngày thiếu: {e}")
    finally:
        if driver:
            print("Đang đóng trình duyệt.")
            driver.quit()

if __name__ == "__main__":
    missing_dates_from_check = [
        "03-02-2025",
        "04-01-2025",
        "07-03-2025",
        "10-03-2025",
        "11-02-2025",
        "14-02-2025",
        "14-03-2025",
        "16-04-2025",
        "17-01-2025",
        "18-04-2025",
        "20-02-2025",
        "25-03-2025",
        "28-02-2025",
        "29-01-2025"
    ]

    target_base_url = "https://cafef.vn/thi-truong-chung-khoan.chn"
    
    css_selector_of_initial_element_to_click = "#admWrapsite > div.main > div.list-section > div > div.list-main > div.noibat_cate > div.list-focus-main > div.firstitem.wp100.clearfix > div > h2 > a"
    
    article_container_css = "#hasWechoice > div" 
    title_css_inside_container = "h3 > a" 
    sapo_css_inside_container = "p.sapo"
    article_url_css_inside_container = "a.avatar.img-resize"
    page_date_css = "#admWrapsite > div.main > div.list-section.list-event > div > div.list-main > h1 > a"

    # Gọi hàm để cào các ngày bị thiếu
    crawl_missing_dates(missing_dates_from_check, 
                        target_base_url, 
                        css_selector_of_initial_element_to_click,
                        article_container_css,
                        title_css_inside_container,
                        sapo_css_inside_container,
                        article_url_css_inside_container,
                        page_date_css)

Đã tải dữ liệu cũ từ 'raw_news_urls.json' để cập nhật.

--- Bắt đầu cào lại 14 ngày bị thiếu ---

--- Đang xử lý ngày thiếu: 3/2/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 2.
Đã chọn Ngày: 3.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 03-02-2025
Tìm thấy 19 khối bài báo.
Đã cào được 18 bài báo cho ngày '03-02-2025'.
Đã cập nhật 'raw_news_urls.json'.

--- Đang xử lý ngày thiếu: 4/1/2025 ---
Đang truy cập: https://cafef.vn/thi-truong-chung-khoan.chn
Đã click vào phần tử ban đầu (nếu cần).
Đã chọn Năm: 2025.
Đã chọn Tháng: 1.
Đã chọn Ngày: 4.
Đang tìm và click nút 'XEM' với selector: a.btn-view.font1
Đã click nút 'XEM'.
Đang cuộn xuống cuối trang để tải hết nội dung...
Đã cuộn đến cuối trang.
Ngày của trang được cào (để làm key JSON): 04-01-

In [25]:
if __name__ == "__main__":
    # Đặt đường dẫn đến file JSON của bạn
    json_file = 'raw_news_urls.json' 
    
    # Đặt khoảng thời gian bạn muốn kiểm tra
    # Current time is Monday, June 23, 2025 at 3:09:47 PM +07.
    # Nên end_date sẽ là 23-06-2025 (Ngày hiện tại)
    start_check_date = '01-01-2025'
    end_check_date = '23-06-2025' 

    find_missing_dates_in_json(json_file, start_check_date, end_check_date)

Số ngày đã cào được trong file JSON: 174
Tổng số ngày dự kiến cần có: 174

Không có ngày nào bị thiếu dữ liệu trong khoảng đã chỉ định.


In [26]:
def navigate_and_select_date_cafef(driver, url, target_day_value, target_month_value, target_year_value, css_selector_to_click_init):
    """
    Truy cập trang web, click phần tử ban đầu, chọn ngày/tháng/năm trong dropdowns,
    và click nút XEM. Driver được truyền vào và trả về để tái sử dụng.
    """
    try:
        if driver.current_url != url or not driver.page_source:
            print(f"Đang truy cập: {url}")
            driver.get(url)
            time.sleep(5)

        try:
            element_init_click = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector_to_click_init))
            )
            element_init_click.click()
            time.sleep(3)
            print("Đã click vào phần tử ban đầu (nếu cần).")
        except Exception:
            pass 

        # Tương tác với dropdown Năm
        select_element_year = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "opNam"))
        )
        select_year = Select(select_element_year)
        select_year.select_by_value(str(target_year_value))
        print(f"Đã chọn Năm: {target_year_value}.")
        time.sleep(1)

        # Tương tác với dropdown Tháng
        select_element_month = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "opThang"))
        )
        select_month = Select(select_element_month)
        select_month.select_by_value(str(target_month_value))
        print(f"Đã chọn Tháng: {target_month_value}.")
        time.sleep(1)

        # Tương tác với dropdown Ngày
        select_element_day = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "opNgay"))
        )
        select_day = Select(select_element_day)
        select_day.select_by_value(str(target_day_value))
        print(f"Đã chọn Ngày: {target_day_value}.")
        time.sleep(1)
        
        # Click vào nút "XEM" để áp dụng bộ lọc ngày
        btn_view_selector = "a.btn-view.font1"
        print(f"Đang tìm và click nút 'XEM' với selector: {btn_view_selector}")
        btn_view = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, btn_view_selector))
        )
        btn_view.click()
        print("Đã click nút 'XEM'.")
        time.sleep(5)

        return driver
    
    except Exception as e:
        print(f"Lỗi trong navigate_and_select_date_cafef: {e}")
        return None

def crawl_article_urls_from_filtered_page(driver_instance, article_container_selector, title_css_inside_container, sapo_css_inside_container, article_url_selector, page_date_selector):
    """
    Cào URL, tiêu đề, mô tả các bài báo từ trang hiện tại của driver sau khi đã lọc.
    Cũng lấy ngày hiển thị trên tiêu đề trang để tổ chức dữ liệu.
    """
    articles_list_for_date = []
    
    # Cuộn xuống cuối trang để tải hết nội dung (nếu có lazy loading)
    print("Đang cuộn xuống cuối trang để tải hết nội dung...")
    last_height = driver_instance.execute_script("return document.body.scrollHeight")
    while True:
        driver_instance.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver_instance.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    print("Đã cuộn đến cuối trang.")

    html_content = driver_instance.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    # Lấy ngày từ tiêu đề trang
    page_date_element = soup.select_one(page_date_selector)
    page_date_text = page_date_element.get_text(strip=True) if page_date_element else "Unknown Date"
    
    date_match = re.search(r'\d{1,2}/\d{1,2}/\d{4}', page_date_text)
    if date_match:
        formatted_date = date_match.group(0).replace('/', '-')
    else:
        formatted_date = page_date_text.replace(':', '').replace('/', '-').replace(' ', '_')
    
    print(f"Ngày của trang được cào (để làm key JSON): {formatted_date}")

    article_containers = soup.select(article_container_selector)
    print(f"Tìm thấy {len(article_containers)} khối bài báo.")

    if not article_containers:
        print("Không tìm thấy bài báo nào với selector đã cho. Vui lòng kiểm tra lại selector.")
        return formatted_date, []

    for i, container in enumerate(article_containers):
        title_element = container.select_one(title_css_inside_container)
        sapo_element = container.select_one(sapo_css_inside_container)
        article_url_element = container.select_one(article_url_selector)

        title = title_element.get_text(strip=True) if title_element else None
        url = article_url_element['href'] if article_url_element and 'href' in article_url_element.attrs else None
        sapo = sapo_element.get_text(strip=True) if sapo_element else None

        if title and url:
            if url and not url.startswith('http'):
                url = f"https://cafef.vn{url}" 
            
            articles_list_for_date.append({
                'Tieu de': title,
                'URL': url,
                'Mo ta': sapo
            })
        else:
            pass # Bỏ qua bài báo thiếu thông tin

    return formatted_date, articles_list_for_date


def crawl_full_article_content(urls_json_file='raw_news_urls.json', output_csv_file='raw_news_cafef.csv', target_month=None, target_year=None):
    """
    Đọc URL từ file JSON (có thể lọc theo tháng/năm), truy cập từng URL để cào nội dung đầy đủ của bài báo (chỉ văn bản),
    và lưu vào file CSV.

    Args:
        urls_json_file (str): Đường dẫn đến file JSON chứa dữ liệu URLs đã cào.
        output_csv_file (str): Đường dẫn đến file CSV để lưu nội dung đầy đủ.
        target_month (int, optional): Tháng cần cào (1-12). Nếu None, cào tất cả các tháng.
        target_year (int, optional): Năm cần cào. Nếu None, cào tất cả các năm.
    """
    
    # 1. Đọc URLs từ file JSON
    if not os.path.exists(urls_json_file):
        print(f"Lỗi: File '{urls_json_file}' không tìm thấy. Vui lòng đảm bảo file tồn tại.")
        return

    all_article_urls_data = {}
    with open(urls_json_file, 'r', encoding='utf-8') as f:
        try:
            all_article_urls_data = json.load(f)
            print(f"Đã tải {len(all_article_urls_data)} ngày dữ liệu URLs từ '{urls_json_file}'.")
        except json.JSONDecodeError:
            print(f"Lỗi: Không thể đọc file JSON '{urls_json_file}'. File có thể bị hỏng hoặc trống.")
            return

    # Lọc dữ liệu theo tháng và năm 
    articles_to_process = {}
    if target_month is not None and target_year is not None:
        print(f"Đang lọc dữ liệu URLs cho tháng {target_month}/{target_year}...")
        for date_key, articles in all_article_urls_data.items():
            try:
                # Phân tích ngày từ date_key (ví dụ: '01-01-2025' hoặc '1-1-2025')
                parts = date_key.split('-')
                if len(parts) == 3:
                    day_json, month_json, year_json = int(parts[0]), int(parts[1]), int(parts[2])
                    if month_json == target_month and year_json == target_year:
                        articles_to_process[date_key] = articles
            except ValueError:
                print(f"Cảnh báo: Định dạng ngày không hợp lệ trong JSON key: {date_key}. Bỏ qua.")
        print(f"Đã lọc được {len(articles_to_process)} ngày dữ liệu cho tháng {target_month}/{target_year}.")
        if not articles_to_process:
            print(f"Không tìm thấy dữ liệu URLs nào cho tháng {target_month}/{target_year}. Kết thúc.")
            return
    else:
        articles_to_process = all_article_urls_data # Nếu không chỉ định tháng/năm, xử lý tất cả

    # 2. Chuẩn bị DataFrame để lưu kết quả
    crawled_full_articles = []
    
    # 3. Khởi tạo WebDriver
    driver = webdriver.Chrome()
    driver.set_page_load_timeout(30) # Đặt timeout cho việc tải trang

    # Các selector cho nội dung bài báo đầy đủ
    sapo_selector_detail = "#admWrapsite > div.main > div.detail-section.adm-mainsection > div:nth-child(1) > div > div.left_cate.totalcontentdetail > div:nth-child(7) > h2"
    content_selector_detail = "#admWrapsite > div.main > div.detail-section.adm-mainsection > div:nth-child(1) > div > div.left_cate.totalcontentdetail > div:nth-child(7) > div.contentdetail"

    # Lấy danh sách các URL đã được cào nội dung để tránh cào lại
    processed_urls = set()
    if os.path.exists(output_csv_file):
        try:
            existing_df = pd.read_csv(output_csv_file, encoding='utf-8-sig')
            processed_urls = set(existing_df['URL'].tolist())
            print(f"Đã tải {len(processed_urls)} URL đã cào nội dung từ '{output_csv_file}'.")
        except Exception as e:
            print(f"Cảnh báo: Không thể đọc file CSV '{output_csv_file}' hoặc file trống. Bắt đầu cào mới. Lỗi: {e}")
            processed_urls = set()

    total_articles_to_crawl_in_selection = sum(len(articles) for articles in articles_to_process.values())
    articles_crawled_count = 0
    
    print(f"\nBắt đầu cào nội dung cho {total_articles_to_crawl_in_selection} bài báo (đã lọc).")

    try:
        # Sắp xếp các ngày để cào theo thứ tự thời gian
        # Đảm bảo datetime đã được import ở đầu file
        sorted_date_keys = sorted(articles_to_process.keys(), 
                                  key=lambda d: datetime.strptime(d, '%d-%m-%Y') if re.match(r'\d{2}-\d{2}-\d{4}', d) else datetime.min)
        
        for date_key in sorted_date_keys:
            articles_for_date = articles_to_process[date_key]
            print(f"\n--- Xử lý các bài báo cho ngày: {date_key} ({len(articles_for_date)} bài) ---")
            for article in articles_for_date:
                url = article.get('URL')
                title = article.get('Tieu de')
                sapo_summary = article.get('Mo ta')

                if not url:
                    print(f"Bỏ qua bài báo thiếu URL: {title}")
                    continue

                if url in processed_urls:
                    articles_crawled_count += 1
                    continue
                
                full_content_text = None
                try:
                    print(f"Đang cào nội dung: {url}")
                    driver.get(url)
                    time.sleep(3)
                    
                    html_content = driver.page_source
                    soup = BeautifulSoup(html_content, 'html.parser')

                    sapo_detail_element = soup.select_one(sapo_selector_detail)
                    sapo_full = sapo_detail_element.get_text(strip=True) if sapo_detail_element else sapo_summary

                    content_element = soup.select_one(content_selector_detail)
                    
                    if content_element:
                        # Loại bỏ các thẻ ảnh và các thẻ không phải văn bản
                        for tag in content_element.find_all(['img', 'script', 'style', 'iframe', 'video']):
                            tag.extract() 
                        
                        full_content_text = content_element.get_text(separator='\n', strip=True)
                        full_content_text = ' '.join(full_content_text.split())

                    if full_content_text:
                        crawled_full_articles.append({
                            'Ngay loc': date_key,
                            'Tieu de': title,
                            'URL': url,
                            'Mo ta tu trang list': sapo_summary,
                            'Sapo chi tiet': sapo_full,
                            'Noi dung day du': full_content_text
                        })
                        processed_urls.add(url)
                        articles_crawled_count += 1
                        print(f"Đã cào nội dung bài: {title}")
                    else:
                        print(f"Không tìm thấy nội dung đầy đủ cho bài: {title} ({url})")

                except Exception as e:
                    print(f"Lỗi khi cào nội dung bài {url}: {e}")
                    articles_crawled_count += 1

                # Lưu DataFrame vào CSV sau mỗi vài bài hoặc mỗi ngày để tránh mất dữ liệu lớn
                if len(crawled_full_articles) > 0 and len(crawled_full_articles) % 10 == 0:
                    pd.DataFrame(crawled_full_articles).to_csv(output_csv_file, mode='a', header=not os.path.exists(output_csv_file), index=False, encoding='utf-8-sig')
                    crawled_full_articles = []
                    print(f"Đã lưu {10} bài báo vào '{output_csv_file}'. Tổng số bài đã cào: {articles_crawled_count}/{total_articles_to_crawl_in_selection}")
                    time.sleep(1)

    except Exception as e:
        print(f"Lỗi tổng thể trong quá trình cào nội dung: {e}")
    finally:
        if driver:
            driver.quit()
        # Lưu nốt các bài còn lại vào CSV
        if len(crawled_full_articles) > 0:
            pd.DataFrame(crawled_full_articles).to_csv(output_csv_file, mode='a', header=not os.path.exists(output_csv_file), index=False, encoding='utf-8-sig')
            print(f"Đã lưu nốt {len(crawled_full_articles)} bài báo vào '{output_csv_file}'.")
        print(f"\nQuá trình cào nội dung hoàn tất. Tổng số bài đã cào: {articles_crawled_count}/{total_articles_to_crawl_in_selection}")

if __name__ == "__main__":
    urls_json_input_file = 'raw_news_urls.json'
    output_articles_csv_file = 'raw_news_cafef.csv'

    month_to_process = 1
    year_to_process = 2025
    
    crawl_full_article_content(urls_json_input_file, 
                               output_articles_csv_file,
                               target_month=month_to_process,
                               target_year=year_to_process)

In [9]:
# --- Chạy chính ---
if __name__ == "__main__":
    urls_json_input_file = 'raw_news_urls.json'
    output_articles_csv_file = 'raw_news_cafef.csv'

    month_to_process = 2
    year_to_process = 2025
    
    crawl_full_article_content(urls_json_input_file, 
                               output_articles_csv_file,
                               target_month=month_to_process,
                               target_year=year_to_process)

Đã tải 174 ngày dữ liệu URLs từ 'raw_news_urls.json'.
Đang lọc dữ liệu URLs cho tháng 2/2025...
Đã lọc được 28 ngày dữ liệu cho tháng 2/2025.
Đã tải 426 URL đã cào nội dung từ 'raw_news_cafef.csv'.

Bắt đầu cào nội dung cho 436 bài báo (đã lọc).

--- Xử lý các bài báo cho ngày: 1-2-2025 (3 bài) ---
Đang cào nội dung: https://cafef.vn/he-lo-muc-luong-khung-cua-phi-cong-cac-hang-bay-viet-nam-188250201112052115.chn
Đã cào nội dung bài: Hé lộ mức lương 'khủng' của phi công các hãng bay Việt Nam
Đang cào nội dung: https://cafef.vn/ba-chu-cai-nao-se-giup-nha-dau-tu-menh-kim-vuot-qua-nam-xung-khac-at-ty-188250131222702214.chn
Đã cào nội dung bài: "Ba chữ cái" nào sẽ giúp nhà đầu tư mệnh Kim vượt qua năm xung khắc Ất Tỵ?
Đang cào nội dung: https://cafef.vn/xu-huong-dong-tien-2025-kenh-dau-tu-nao-se-len-ngoi-188250131220824105.chn
Đã cào nội dung bài: Xu hướng dòng tiền 2025: Kênh đầu tư nào sẽ lên ngôi?

--- Xử lý các bài báo cho ngày: 2-2-2025 (6 bài) ---
Đang cào nội dung: https://cafef.vn/b

In [10]:
if __name__ == "__main__":
    urls_json_input_file = 'raw_news_urls.json'
    output_articles_csv_file = 'raw_news_cafef.csv'
    
    month_to_process = 3
    year_to_process = 2025
    
    crawl_full_article_content(urls_json_input_file, 
                               output_articles_csv_file,
                               target_month=month_to_process,
                               target_year=year_to_process)

Đã tải 174 ngày dữ liệu URLs từ 'raw_news_urls.json'.
Đang lọc dữ liệu URLs cho tháng 3/2025...
Đã lọc được 31 ngày dữ liệu cho tháng 3/2025.
Đã tải 862 URL đã cào nội dung từ 'raw_news_cafef.csv'.

Bắt đầu cào nội dung cho 499 bài báo (đã lọc).

--- Xử lý các bài báo cho ngày: 1-3-2025 (10 bài) ---
Đang cào nội dung: https://cafef.vn/pho-chu-tich-bamboo-land-xin-tu-nhiem-188250301201735828.chn
Đã cào nội dung bài: Phó Chủ tịch Bamboo Land xin từ nhiệm
Đang cào nội dung: https://cafef.vn/mien-phi-truoc-ba-o-to-dien-den-nam-2027-188250301165455971.chn
Đã cào nội dung bài: Miễn phí trước bạ ô tô điện đến năm 2027
Đang cào nội dung: https://cafef.vn/thu-tuong-new-zealand-cung-ty-phu-nguyen-thi-phuong-thao-vua-lam-1-viec-dac-biet-tren-may-bay-vietjet-188250301121238978.chn
Đã cào nội dung bài: Thủ tướng New Zealand cùng tỷ phú Nguyễn Thị Phương Thảo vừa làm 1 việc đặc biệt trên máy bay Vietjet
Đang cào nội dung: https://cafef.vn/them-mot-lanh-dao-cap-cao-trong-he-sinh-thai-bamboo-capital-b

In [11]:
# --- Chạy chính ---
if __name__ == "__main__":
    urls_json_input_file = 'raw_news_urls.json'
    output_articles_csv_file = 'raw_news_cafef.csv'

    month_to_process = 4
    year_to_process = 2025
    
    crawl_full_article_content(urls_json_input_file, 
                               output_articles_csv_file,
                               target_month=month_to_process,
                               target_year=year_to_process)

Đã tải 174 ngày dữ liệu URLs từ 'raw_news_urls.json'.
Đang lọc dữ liệu URLs cho tháng 4/2025...
Đã lọc được 30 ngày dữ liệu cho tháng 4/2025.
Đã tải 1133 URL đã cào nội dung từ 'raw_news_cafef.csv'.

Bắt đầu cào nội dung cho 470 bài báo (đã lọc).

--- Xử lý các bài báo cho ngày: 1-4-2025 (19 bài) ---
Đang cào nội dung: https://cafef.vn/chung-khoan-ngay-mai-2-4-vn-index-thoat-hiem-gio-chot-nhip-chinh-da-ket-thuc-188250401205059461.chn
Đã cào nội dung bài: Chứng khoán ngày mai, 2-4: VN-Index "thoát hiểm" giờ chót, nhịp chỉnh đã kết thúc?
Đang cào nội dung: https://cafef.vn/hai-co-phieu-duoc-tu-doanh-ctck-gom-hang-tram-ty-trong-phien-dau-thang-188250401181646732.chn
Đã cào nội dung bài: Hai cổ phiếu được tự doanh CTCK "gom" hàng trăm tỷ trong phiên đầu tháng
Đang cào nội dung: https://cafef.vn/hang-taxi-cua-ong-pham-nhat-vuong-choi-lon-chi-hon-16-ty-cho-khach-hang-va-tai-xe-moi-tuan-tang-1-xe-dien-vf3-188250401175805257.chn
Đã cào nội dung bài: Hãng taxi của ông Phạm Nhật Vượng “chơi lớn”

In [3]:
# --- Chạy chính ---
if __name__ == "__main__":
    urls_json_input_file = 'raw_news_urls.json'
    output_articles_csv_file = 'raw_news_cafef.csv'
 
    month_to_process = 5
    year_to_process = 2025
    
    crawl_full_article_content(urls_json_input_file, 
                               output_articles_csv_file,
                               target_month=month_to_process,
                               target_year=year_to_process)

Đã tải 174 ngày dữ liệu URLs từ 'raw_news_urls.json'.
Đang lọc dữ liệu URLs cho tháng 5/2025...
Đã lọc được 31 ngày dữ liệu cho tháng 5/2025.
Đã tải 1598 URL đã cào nội dung từ 'raw_news_cafef.csv'.

Bắt đầu cào nội dung cho 489 bài báo (đã lọc).

--- Xử lý các bài báo cho ngày: 1-5-2025 (10 bài) ---
Đang cào nội dung: https://cafef.vn/vietnam-airlines-bao-lai-hon-3600-ty-dong-trong-quy-i-2025-nho-khach-quoc-te-va-gia-nhien-lieu-188250501105553159.chn
Đã cào nội dung bài: Vietnam Airlines báo lãi hơn 3.600 tỷ đồng trong quý I/2025 nhờ khách quốc tế và giá nhiên liệu
Đang cào nội dung: https://cafef.vn/becamex-idc-lai-quy-i-2025-gap-3-lan-cung-ky-vay-no-tai-chinh-tren-23000-ty-dong-188250501104605608.chn
Đã cào nội dung bài: Becamex IDC lãi quý I/2025 gấp 3 lần cùng kỳ, vay nợ tài chính trên 23.000 tỷ đồng
Đang cào nội dung: https://cafef.vn/thi-truong-nghi-le-cong-ty-chung-khoan-tang-toc-chuyen-doi-sang-krx-188250501102139229.chn
Đã cào nội dung bài: Thị trường nghỉ lễ, công ty chứng k

In [4]:
# --- Chạy chính ---
if __name__ == "__main__":
    urls_json_input_file = 'raw_news_urls.json'
    output_articles_csv_file = 'raw_news_cafef.csv'
    
    month_to_process = 6
    year_to_process = 2025

    crawl_full_article_content(urls_json_input_file, 
                               output_articles_csv_file,
                               target_month=month_to_process,
                               target_year=year_to_process)

Đã tải 174 ngày dữ liệu URLs từ 'raw_news_urls.json'.
Đang lọc dữ liệu URLs cho tháng 6/2025...
Đã lọc được 23 ngày dữ liệu cho tháng 6/2025.
Đã tải 2085 URL đã cào nội dung từ 'raw_news_cafef.csv'.

Bắt đầu cào nội dung cho 348 bài báo (đã lọc).

--- Xử lý các bài báo cho ngày: 1-6-2025 (7 bài) ---
Đang cào nội dung: https://cafef.vn/qua-thang-5-ruc-ro-chung-khoan-thang-6-se-ra-sao-188250601211245904.chn
Đã cào nội dung bài: Qua tháng 5 rực rỡ, chứng khoán tháng 6 sẽ ra sao?
Đang cào nội dung: https://cafef.vn/goc-nhin-chuyen-gia-nhip-dieu-chinh-la-co-hoi-giai-ngan-2-nhom-nganh-duoc-dong-tien-chu-y-188250601154313573.chn
Đã cào nội dung bài: Góc nhìn chuyên gia: Nhịp điều chỉnh là cơ hội giải ngân 2 nhóm ngành được dòng tiền chú ý
Đang cào nội dung: https://cafef.vn/thu-tuong-vietnam-airlines-gop-phan-nang-vi-the-quoc-gia-tren-truong-quoc-te-188250601152923478.chn
Đã cào nội dung bài: Thủ tướng: Vietnam Airlines góp phần nâng vị thế quốc gia trên trường quốc tế
Đang cào nội dung: http

Crawl các bài báo còn thiếu

In [6]:
def crawl_single_article_content(driver_instance, url, title, sapo_summary, date_key,
                                 sapo_selector_detail, content_selector_detail):
    """
    Cào nội dung đầy đủ của MỘT bài báo từ URL đã cho.
    Trả về dictionary chứa thông tin bài báo hoặc None nếu lỗi.
    """
    full_content_text = None
    sapo_full = sapo_summary 

    try:
        print(f"Đang cào nội dung: {url}")
        driver_instance.get(url)
        time.sleep(3) # Đợi trang tải xong nội dung
        
        html_content = driver_instance.page_source
        soup = BeautifulSoup(html_content, 'html.parser')

        # Trích xuất sapo chi tiết
        sapo_detail_element = soup.select_one(sapo_selector_detail)
        sapo_full = sapo_detail_element.get_text(strip=True) if sapo_detail_element else sapo_summary

        # Trích xuất nội dung chính
        content_element = soup.select_one(content_selector_detail)
        
        if content_element:
            # Loại bỏ các thẻ ảnh và các thẻ không phải văn bản
            for tag in content_element.find_all(['img', 'script', 'style', 'iframe', 'video']):
                tag.extract() 
            
            full_content_text = content_element.get_text(separator='\n', strip=True)
            full_content_text = ' '.join(full_content_text.split())

        if full_content_text:
            print(f"Đã cào nội dung bài: {title}")
            return {
                'Ngay loc': date_key,
                'Tieu de': title,
                'URL': url,
                'Mo ta tu trang list': sapo_summary,
                'Sapo chi tiet': sapo_full,
                'Noi dung day du': full_content_text
            }
        else:
            print(f"Không tìm thấy nội dung đầy đủ cho bài: {title} ({url})")
            return None

    except Exception as e:
        print(f"Lỗi khi cào nội dung bài {url}: {e}")
        return None
# Hàm chính để kiểm tra và cào lại
def re_crawl_missing_article_contents(urls_json_file='raw_news_urls.json', output_csv_file='raw_news_cafef.csv'):
    """
    Kiểm tra các URL trong file JSON so với các URL đã có nội dung trong file CSV.
    Cào lại nội dung cho các URL bị thiếu và cập nhật vào file CSV.
    """
    
    # 1. Đọc tất cả URLs dự kiến từ file JSON
    if not os.path.exists(urls_json_file):
        print(f"Lỗi: File '{urls_json_file}' không tìm thấy. Vui lòng đảm bảo file tồn tại.")
        return

    all_expected_urls_data = {} # Dữ liệu từ JSON
    with open(urls_json_file, 'r', encoding='utf-8') as f:
        try:
            all_expected_urls_data = json.load(f)
            print(f"Đã tải dữ liệu URLs từ '{urls_json_file}'.")
        except json.JSONDecodeError:
            print(f"Lỗi: Không thể đọc file JSON '{urls_json_file}'. File có thể bị hỏng hoặc trống.")
            return

    # Chuẩn hóa tất cả các URL và thông tin liên quan từ JSON thành một list phẳng để dễ xử lý
    expected_articles_flat = []
    for date_key, articles_for_date in all_expected_urls_data.items():
        for article in articles_for_date:
            url = article.get('URL')
            if url: # Chỉ thêm URL hợp lệ
                # Chuẩn hóa URL nếu nó là tương đối
                if not url.startswith('http'):
                    url = f"https://cafef.vn{url}"
                expected_articles_flat.append({
                    'URL': url,
                    'Tieu de': article.get('Tieu de'),
                    'Mo ta': article.get('Mo ta'),
                    'Ngay loc': date_key
                })
    expected_urls_set = {item['URL'] for item in expected_articles_flat}
    print(f"Tổng số URL dự kiến từ JSON: {len(expected_urls_set)}")


    # 2. Đọc tất cả URLs đã có nội dung từ file CSV
    crawled_urls_in_csv = set()
    if os.path.exists(output_csv_file):
        try:
            existing_df = pd.read_csv(output_csv_file, encoding='utf-8-sig')
            if 'URL' in existing_df.columns:
                crawled_urls_in_csv = set(existing_df['URL'].tolist())
            else:
                print(f"Cảnh báo: File CSV '{output_csv_file}' không có cột 'URL'.")
        except pd.errors.EmptyDataError:
            print(f"File CSV '{output_csv_file}' trống.")
        except Exception as e:
            print(f"Lỗi khi đọc file CSV '{output_csv_file}': {e}")
    print(f"Tổng số URL đã cào nội dung trong CSV: {len(crawled_urls_in_csv)}")

    # 3. Tìm các URL bị thiếu nội dung
    missing_urls_to_crawl = []
    for article_info in expected_articles_flat:
        if article_info['URL'] not in crawled_urls_in_csv:
            missing_urls_to_crawl.append(article_info)
    
    # Sắp xếp các URL bị thiếu theo ngày để cào theo thứ tự
    missing_urls_to_crawl.sort(key=lambda x: datetime.strptime(x['Ngay loc'], '%d-%m-%Y') if re.match(r'\d{1,2}-\d{1,2}-\d{4}', x['Ngay loc']) else datetime.min)

    print(f"\nTìm thấy {len(missing_urls_to_crawl)} bài báo cần cào lại nội dung.")

    if not missing_urls_to_crawl:
        print("Không có bài báo nào bị thiếu nội dung. Quá trình hoàn tất.")
        return

    # 4. Cào lại các URL bị thiếu
    driver = webdriver.Chrome()
    driver.set_page_load_timeout(30) # Đặt timeout cho việc tải trang

    # Các selector cho nội dung bài báo đầy đủ
    sapo_selector_detail = "#admWrapsite > div.main > div.detail-section.adm-mainsection > div:nth-child(1) > div > div.left_cate.totalcontentdetail > div:nth-child(7) > h2"
    content_selector_detail = "#admWrapsite > div.main > div.detail-section.adm-mainsection > div:nth-child(1) > div > div.left_cate.totalcontentdetail > div:nth-child(7) > div.contentdetail"

    articles_to_append = [] # List tạm thời để lưu các bài báo mới cào được

    try:
        for i, article_info in enumerate(missing_urls_to_crawl):
            url = article_info['URL']
            title = article_info['Tieu de']
            sapo_summary = article_info['Mo ta']
            date_key = article_info['Ngay loc']

            print(f"\n--- Đang cào lại bài {i+1}/{len(missing_urls_to_crawl)}: {title} ({url}) ---")
            crawled_data = crawl_single_article_content(driver, url, title, sapo_summary, date_key,
                                                          sapo_selector_detail, content_selector_detail)
            
            if crawled_data:
                articles_to_append.append(crawled_data)
            
            # Lưu vào CSV định kỳ để tránh mất dữ liệu
            if len(articles_to_append) > 0 and len(articles_to_append) % 10 == 0:
                pd.DataFrame(articles_to_append).to_csv(output_csv_file, mode='a', header=not os.path.exists(output_csv_file), index=False, encoding='utf-8-sig')
                articles_to_append = [] # Xóa list để giải phóng bộ nhớ
                print(f"Đã lưu {10} bài báo mới vào '{output_csv_file}'.")
                time.sleep(1) # Nghỉ một chút sau khi lưu

    except Exception as e:
        print(f"Lỗi tổng thể trong quá trình cào lại: {e}")
    finally:
        if driver:
            driver.quit()
        # Lưu nốt các bài còn lại vào CSV
        if len(articles_to_append) > 0:
            pd.DataFrame(articles_to_append).to_csv(output_csv_file, mode='a', header=not os.path.exists(output_csv_file), index=False, encoding='utf-8-sig')
            print(f"Đã lưu nốt {len(articles_to_append)} bài báo mới vào '{output_csv_file}'.")
        print("\nQuá trình cào lại các bài báo thiếu hoàn tất.")

if __name__ == "__main__":
    urls_json_input_file = 'raw_news_urls.json'
    output_articles_csv_file = 'raw_news_cafef.csv'
    
    re_crawl_missing_article_contents(urls_json_input_file, output_articles_csv_file)

Đã tải dữ liệu URLs từ 'raw_news_urls.json'.
Tổng số URL dự kiến từ JSON: 2668
Tổng số URL đã cào nội dung trong CSV: 2429

Tìm thấy 239 bài báo cần cào lại nội dung.

--- Đang cào lại bài 1/239: Ngân hàng số 1 Việt Nam chính thức có Tổng Giám đốc mới (https://cafef.vn/ngan-hang-so-1-viet-nam-chinh-thuc-co-tong-giam-doc-moi-188250307193624531.chn) ---
Đang cào nội dung: https://cafef.vn/ngan-hang-so-1-viet-nam-chinh-thuc-co-tong-giam-doc-moi-188250307193624531.chn
Lỗi khi cào nội dung bài https://cafef.vn/ngan-hang-so-1-viet-nam-chinh-thuc-co-tong-giam-doc-moi-188250307193624531.chn: Message: timeout: Timed out receiving message from renderer: 29.799
  (Session info: chrome=137.0.7151.120)
Stacktrace:
	GetHandleVerifier [0x0x7ff7d921cda5+78885]
	GetHandleVerifier [0x0x7ff7d921ce00+78976]
	(No symbol) [0x0x7ff7d8fd9bca]
	(No symbol) [0x0x7ff7d8fc706c]
	(No symbol) [0x0x7ff7d8fc6d5a]
	(No symbol) [0x0x7ff7d8fc492f]
	(No symbol) [0x0x7ff7d8fc538f]
	(No symbol) [0x0x7ff7d8fd402e]
	(No symb

In [7]:
if __name__ == "__main__":
    urls_json_input_file = 'raw_news_urls.json'
    output_articles_csv_file = 'raw_news_cafef.csv'
    
    re_crawl_missing_article_contents(urls_json_input_file, output_articles_csv_file)

Đã tải dữ liệu URLs từ 'raw_news_urls.json'.
Tổng số URL dự kiến từ JSON: 2668
Tổng số URL đã cào nội dung trong CSV: 2635

Tìm thấy 33 bài báo cần cào lại nội dung.

--- Đang cào lại bài 1/33: Ngân hàng số 1 Việt Nam chính thức có Tổng Giám đốc mới (https://cafef.vn/ngan-hang-so-1-viet-nam-chinh-thuc-co-tong-giam-doc-moi-188250307193624531.chn) ---
Đang cào nội dung: https://cafef.vn/ngan-hang-so-1-viet-nam-chinh-thuc-co-tong-giam-doc-moi-188250307193624531.chn
Đã cào nội dung bài: Ngân hàng số 1 Việt Nam chính thức có Tổng Giám đốc mới

--- Đang cào lại bài 2/33: Trong lúc chứng khoán đang ở đỉnh, tôi lại miệt mài trung bình giá cổ phiếu trong danh mục, nhìn sang giá vàng tăng lên 98 triệu càng làm tôi buồn hơn (https://cafef.vn/trong-luc-chung-khoan-dang-o-dinh-toi-lai-miet-mai-trung-binh-gia-co-phieu-trong-danh-muc-nhin-sang-gia-vang-tang-len-98-trieu-cang-lam-toi-buon-hon-188250318135425134.chn) ---
Đang cào nội dung: https://cafef.vn/trong-luc-chung-khoan-dang-o-dinh-toi-lai-miet

In [8]:
# --- Chạy chính ---
if __name__ == "__main__":
    urls_json_input_file = 'raw_news_urls.json'
    output_articles_csv_file = 'raw_news_cafef.csv'
    
    re_crawl_missing_article_contents(urls_json_input_file, output_articles_csv_file)

Đã tải dữ liệu URLs từ 'raw_news_urls.json'.
Tổng số URL dự kiến từ JSON: 2668
Tổng số URL đã cào nội dung trong CSV: 2662

Tìm thấy 6 bài báo cần cào lại nội dung.

--- Đang cào lại bài 1/6: Trong lúc chứng khoán đang ở đỉnh, tôi lại miệt mài trung bình giá cổ phiếu trong danh mục, nhìn sang giá vàng tăng lên 98 triệu càng làm tôi buồn hơn (https://cafef.vn/trong-luc-chung-khoan-dang-o-dinh-toi-lai-miet-mai-trung-binh-gia-co-phieu-trong-danh-muc-nhin-sang-gia-vang-tang-len-98-trieu-cang-lam-toi-buon-hon-188250318135425134.chn) ---
Đang cào nội dung: https://cafef.vn/trong-luc-chung-khoan-dang-o-dinh-toi-lai-miet-mai-trung-binh-gia-co-phieu-trong-danh-muc-nhin-sang-gia-vang-tang-len-98-trieu-cang-lam-toi-buon-hon-188250318135425134.chn
Không tìm thấy nội dung đầy đủ cho bài: Trong lúc chứng khoán đang ở đỉnh, tôi lại miệt mài trung bình giá cổ phiếu trong danh mục, nhìn sang giá vàng tăng lên 98 triệu càng làm tôi buồn hơn (https://cafef.vn/trong-luc-chung-khoan-dang-o-dinh-toi-lai-mie

In [32]:
if __name__ == "__main__":
    urls_json_input_file = 'raw_news_urls.json'
    output_articles_csv_file = 'raw_news_cafef.csv'
    
    re_crawl_missing_article_contents(urls_json_input_file, output_articles_csv_file)

Đã tải dữ liệu URLs từ 'raw_news_urls.json'.
Tổng số URL dự kiến từ JSON: 2668
Tổng số URL đã cào nội dung trong CSV: 2663

Tìm thấy 5 bài báo cần cào lại nội dung.

--- Đang cào lại bài 1/5: Trong lúc chứng khoán đang ở đỉnh, tôi lại miệt mài trung bình giá cổ phiếu trong danh mục, nhìn sang giá vàng tăng lên 98 triệu càng làm tôi buồn hơn (https://cafef.vn/trong-luc-chung-khoan-dang-o-dinh-toi-lai-miet-mai-trung-binh-gia-co-phieu-trong-danh-muc-nhin-sang-gia-vang-tang-len-98-trieu-cang-lam-toi-buon-hon-188250318135425134.chn) ---
Đang cào nội dung: https://cafef.vn/trong-luc-chung-khoan-dang-o-dinh-toi-lai-miet-mai-trung-binh-gia-co-phieu-trong-danh-muc-nhin-sang-gia-vang-tang-len-98-trieu-cang-lam-toi-buon-hon-188250318135425134.chn
Không tìm thấy nội dung đầy đủ cho bài: Trong lúc chứng khoán đang ở đỉnh, tôi lại miệt mài trung bình giá cổ phiếu trong danh mục, nhìn sang giá vàng tăng lên 98 triệu càng làm tôi buồn hơn (https://cafef.vn/trong-luc-chung-khoan-dang-o-dinh-toi-lai-mie

In [33]:
# Hàm kiểm tra và cào lại
def re_crawl_missing_article_contents(urls_json_file='raw_news_urls.json', output_csv_file='raw_news_cafef.csv'):
    """
    Kiểm tra các URL trong file JSON so với các URL đã có nội dung trong file CSV.
    Cào lại nội dung cho các URL bị thiếu và cập nhật vào file CSV.
    """
    
    # 1. Đọc tất cả URLs dự kiến từ file JSON
    if not os.path.exists(urls_json_file):
        print(f"Lỗi: File '{urls_json_file}' không tìm thấy. Vui lòng đảm bảo file tồn tại.")
        return

    all_expected_urls_data = {} # Dữ liệu từ JSON
    with open(urls_json_file, 'r', encoding='utf-8') as f:
        try:
            all_expected_urls_data = json.load(f)
            print(f"Đã tải dữ liệu URLs từ '{urls_json_file}'.")
        except json.JSONDecodeError:
            print(f"Lỗi: Không thể đọc file JSON '{urls_json_file}'. File có thể bị hỏng hoặc trống.")
            return

    # Chuẩn hóa tất cả các URL và thông tin liên quan từ JSON thành một list phẳng để dễ xử lý
    expected_articles_flat = []
    for date_key, articles_for_date in all_expected_urls_data.items():
        for article in articles_for_date:
            url = article.get('URL')
            if url: # Chỉ thêm URL hợp lệ
                # Chuẩn hóa URL nếu nó là tương đối
                if not url.startswith('http'):
                    url = f"https://cafef.vn{url}"
                expected_articles_flat.append({
                    'URL': url,
                    'Tieu de': article.get('Tieu de'),
                    'Mo ta': article.get('Mo ta'),
                    'Ngay loc': date_key
                })
    expected_urls_set = {item['URL'] for item in expected_articles_flat}
    print(f"Tổng số URL dự kiến từ JSON: {len(expected_urls_set)}")


    # 2. Đọc tất cả URLs đã có nội dung từ file CSV
    crawled_urls_in_csv = set()
    if os.path.exists(output_csv_file):
        try:
            existing_df = pd.read_csv(output_csv_file, encoding='utf-8-sig')
            if 'URL' in existing_df.columns:
                crawled_urls_in_csv = set(existing_df['URL'].tolist())
            else:
                print(f"Cảnh báo: File CSV '{output_csv_file}' không có cột 'URL'.")
        except pd.errors.EmptyDataError:
            print(f"File CSV '{output_csv_file}' trống.")
        except Exception as e:
            print(f"Lỗi khi đọc file CSV '{output_csv_file}': {e}")
    print(f"Tổng số URL đã cào nội dung trong CSV: {len(crawled_urls_in_csv)}")

    # 3. Tìm các URL bị thiếu nội dung
    missing_urls_to_crawl = []
    for article_info in expected_articles_flat:
        if article_info['URL'] not in crawled_urls_in_csv:
            missing_urls_to_crawl.append(article_info)
    
    # Sắp xếp các URL bị thiếu theo ngày để cào theo thứ tự
    missing_urls_to_crawl.sort(key=lambda x: datetime.strptime(x['Ngay loc'], '%d-%m-%Y') if re.match(r'\d{1,2}-\d{1,2}-\d{4}', x['Ngay loc']) else datetime.min)

    print(f"\nTìm thấy {len(missing_urls_to_crawl)} bài báo cần cào lại nội dung.")

    if not missing_urls_to_crawl:
        print("Không có bài báo nào bị thiếu nội dung. Quá trình hoàn tất.")
        return

    # 4. Cào lại các URL bị thiếu
    driver = webdriver.Chrome()
    driver.set_page_load_timeout(30) # Đặt timeout cho việc tải trang

    # Cập nhật các selector theo thay đổi mới
    sapo_selector_detail = "#admWrapsite > div.main > div.detail-section.adm-mainsection > div:nth-child(1) > div > div.left_cate.totalcontentdetail > div:nth-child(8) > h2"
    content_selector_detail = "#admWrapsite > div.main > div.detail-section.adm-mainsection > div:nth-child(1) > div > div.left_cate.totalcontentdetail > div:nth-child(8) > div.contentdetail"

    articles_to_append = [] # List tạm thời để lưu các bài báo mới cào được

    try:
        for i, article_info in enumerate(missing_urls_to_crawl):
            url = article_info['URL']
            title = article_info['Tieu de']
            sapo_summary = article_info['Mo ta']
            date_key = article_info['Ngay loc']

            print(f"\n--- Đang cào lại bài {i+1}/{len(missing_urls_to_crawl)}: {title} ({url}) ---")
            crawled_data = crawl_single_article_content(driver, url, title, sapo_summary, date_key,
                                                          sapo_selector_detail, content_selector_detail)
            
            if crawled_data:
                articles_to_append.append(crawled_data)
            
            # Lưu vào CSV định kỳ để tránh mất dữ liệu
            if len(articles_to_append) > 0 and len(articles_to_append) % 10 == 0:
                pd.DataFrame(articles_to_append).to_csv(output_csv_file, mode='a', header=not os.path.exists(output_csv_file), index=False, encoding='utf-8-sig')
                articles_to_append = [] # Xóa list để giải phóng bộ nhớ
                print(f"Đã lưu {10} bài báo mới vào '{output_csv_file}'.")
                time.sleep(1) # Nghỉ một chút sau khi lưu

    except Exception as e:
        print(f"Lỗi tổng thể trong quá trình cào lại: {e}")
    finally:
        if driver:
            driver.quit()
        # Lưu nốt các bài còn lại vào CSV
        if len(articles_to_append) > 0:
            pd.DataFrame(articles_to_append).to_csv(output_csv_file, mode='a', header=not os.path.exists(output_csv_file), index=False, encoding='utf-8-sig')
            print(f"Đã lưu nốt {len(articles_to_append)} bài báo mới vào '{output_csv_file}'.")
        print("\nQuá trình cào lại các bài báo thiếu hoàn tất.")

if __name__ == "__main__":
    urls_json_input_file = 'raw_news_urls.json'
    output_articles_csv_file = 'raw_news_cafef.csv'
    
    re_crawl_missing_article_contents(urls_json_input_file, output_articles_csv_file)

Đã tải dữ liệu URLs từ 'raw_news_urls.json'.
Tổng số URL dự kiến từ JSON: 2668
Tổng số URL đã cào nội dung trong CSV: 2663

Tìm thấy 5 bài báo cần cào lại nội dung.

--- Đang cào lại bài 1/5: Trong lúc chứng khoán đang ở đỉnh, tôi lại miệt mài trung bình giá cổ phiếu trong danh mục, nhìn sang giá vàng tăng lên 98 triệu càng làm tôi buồn hơn (https://cafef.vn/trong-luc-chung-khoan-dang-o-dinh-toi-lai-miet-mai-trung-binh-gia-co-phieu-trong-danh-muc-nhin-sang-gia-vang-tang-len-98-trieu-cang-lam-toi-buon-hon-188250318135425134.chn) ---
Đang cào nội dung: https://cafef.vn/trong-luc-chung-khoan-dang-o-dinh-toi-lai-miet-mai-trung-binh-gia-co-phieu-trong-danh-muc-nhin-sang-gia-vang-tang-len-98-trieu-cang-lam-toi-buon-hon-188250318135425134.chn
Đã cào nội dung bài: Trong lúc chứng khoán đang ở đỉnh, tôi lại miệt mài trung bình giá cổ phiếu trong danh mục, nhìn sang giá vàng tăng lên 98 triệu càng làm tôi buồn hơn

--- Đang cào lại bài 2/5: 10 năm “đánh chứng” gồng lỗ: Trả giá vì sai lầm ai cũng

In [34]:
def check_url_consistency(json_file_path='raw_news_urls.json', csv_file_path='raw_news_cafef.csv'):
    """
    Kiểm tra xem tập hợp các URL trong file JSON và file CSV có giống nhau 100% hay không.
    In ra các URL bị thiếu ở mỗi bên nếu có.
    """
    
    # 1. Đọc tất cả URLs từ file JSON
    json_urls = set()
    if os.path.exists(json_file_path):
        with open(json_file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                for date_key, articles_for_date in data.items():
                    for article in articles_for_date:
                        url = article.get('URL')
                        if url:
                            # Chuẩn hóa URL: thêm domain nếu là tương đối
                            if not url.startswith('http'):
                                url = f"https://cafef.vn{url}"
                            json_urls.add(url)
            except json.JSONDecodeError:
                print(f"Lỗi: Không thể đọc file JSON '{json_file_path}'. File có thể bị hỏng hoặc trống.")
                return False
    else:
        print(f"Lỗi: File JSON '{json_file_path}' không tìm thấy.")
        return False

    print(f"Tổng số URL duy nhất trong JSON: {len(json_urls)}")

    # 2. Đọc tất cả URLs từ file CSV
    csv_urls = set()
    if os.path.exists(csv_file_path):
        try:
            df = pd.read_csv(csv_file_path, encoding='utf-8-sig')
            if 'URL' in df.columns:
                # Đảm bảo URL trong CSV cũng được chuẩn hóa nếu cần, mặc dù thường thì URL đầy đủ đã được lưu
                for url in df['URL'].tolist():
                    if url:
                        if not url.startswith('http'): # phòng trường hợp CSV lưu URL tương đối
                            url = f"https://cafef.vn{url}"
                        csv_urls.add(url)
            else:
                print(f"Cảnh báo: File CSV '{csv_file_path}' không có cột 'URL'.")
        except pd.errors.EmptyDataError:
            print(f"File CSV '{csv_file_path}' trống.")
        except Exception as e:
            print(f"Lỗi khi đọc file CSV '{csv_file_path}': {e}")
    else:
        print(f"Lỗi: File CSV '{csv_file_path}' không tìm thấy.")
        return False

    print(f"Tổng số URL duy nhất trong CSV: {len(csv_urls)}")

    # 3. So sánh hai tập hợp
    missing_in_csv = json_urls - csv_urls
    missing_in_json = csv_urls - json_urls

    if not missing_in_csv and not missing_in_json:
        print("\nKết quả: Các tập hợp URL trong JSON và CSV KHỚP NHAU 100%!")
        return True
    else:
        print("\nKết quả: Các tập hợp URL trong JSON và CSV KHÔNG KHỚP NHAU.")
        
        if missing_in_csv:
            print(f"\n--- URL có trong JSON nhưng KHÔNG có trong CSV ({len(missing_in_csv)} URL): ---")
            for url in sorted(list(missing_in_csv)):
                print(url)
        
        if missing_in_json:
            print(f"\n--- URL có trong CSV nhưng KHÔNG có trong JSON ({len(missing_in_json)} URL): ---")
            for url in sorted(list(missing_in_json)):
                print(url)
        return False

if __name__ == "__main__":
    json_source_file = 'raw_news_urls.json'
    csv_target_file = 'raw_news_cafef.csv'
    
    check_url_consistency(json_source_file, csv_target_file)

Tổng số URL duy nhất trong JSON: 2668
Tổng số URL duy nhất trong CSV: 2667

Kết quả: Các tập hợp URL trong JSON và CSV KHÔNG KHỚP NHAU.

--- URL có trong JSON nhưng KHÔNG có trong CSV (1 URL): ---
https://cafef.vn/vpbanks-va-hanh-trinh-vuon-minh-thanh-dinh-che-chung-khoan-cong-nghe-18825052608023672.chn
